In [ ]:
import os
import numpy as np
import string
import math

from sklearn import model_selection
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

In [ ]:
X_train = [] # an element of X is represented as (filename,text)
Y_train = [] # an element of Y represents the newsgroup category of the corresponding X element
for category in os.listdir('train'):
    for document in os.listdir('train/'+category):
        with open('train/'+category+'/'+document, "r") as f:
            X_train.append((document,f.read()))
            Y_train.append(category)
            
num_category = 2 #Our task is a 2 category classification task           

### Train has 0: rec.autos and 1: talk.politics.guns 
### Test has 0: rec.motorcycles and 1: talk.politics.misc

## Category mapping is 0: Rec and 1:Talk

In [ ]:
Y_train

In [ ]:
X_test = [] # an element of X is represented as (filename,text)
Y_test = [] # an element of Y represents the newsgroup category of the corresponding X element
for category in os.listdir('test'):
    for document in os.listdir('test/'+category):
        with open('test/'+category+'/'+document, "r") as f:
            X_test.append((document,f.read()))
            Y_test.append(category)

In [ ]:
Y_test

In [ ]:
# A list of common english words which should not affect predictions
stopwords = ['those', 'on', 'own', 'yourselves', 'ie', 'around', 'between', 'four', 'been', 'alone', 'off', 'am', 'then', 'other',
             'can', 'cry', 'hereafter', 'front', 'too', 'wherein', 'everything', 'up', 'onto', 'never', 'either', 'how', 'before', 
             'anyway', 'since', 'through', 'amount', 'now', 'he', 'cant', 'was', 'con', 'have', 'into', 'because', 'inc', 'not', 
             'therefore', 'they', 'even', 'whom', 'it', 'see', 'somewhere', 'interest', 'thereupon', 'nothing', 'thick', 'whereas', 
             'much', 'whenever', 'find', 'seem', 'until', 'whereby', 'at', 'ltd', 'fire', 'also', 'some', 'last', 'than', 'get', 
             'already', 'our', 'once', 'will', 'noone', 'that', 'what', 'thus', 'no', 'myself', 'out', 'next', 'whatever', 'although', 
             'though', 'etc', 'which', 'would', 'therein', 'nor', 'somehow', 'whereupon', 'besides', 'whoever', 'thin', 'ourselves', 
             'few', 'third', 'without', 'anything', 'twelve', 'against', 'while', 'twenty', 'if', 'however', 'found', 'herself', 
             'when', 'may', 'ours', 'six', 'done', 'seems', 'else', 'call', 'perhaps', 'had', 'nevertheless', 'fill', 'where', 
             'otherwise', 'still', 'within', 'its', 'for', 'together', 'elsewhere', 'throughout', 'of', 'eg', 'others', 'show', 
             'sincere', 'anywhere', 'anyhow', 'as', 'are', 'the', 'hence', 'something', 'hereby', 'nowhere', 'de', 'latterly', 
             'neither', 'his', 'go', 'forty', 'put', 'their', 'by', 'namely', 'could', 'five', 'itself', 'is', 'nine', 'whereafter', 
             'down', 'bottom', 'thereby', 'such', 'both', 'she', 'become', 'whole', 'who', 'yourself', 'every', 'thru', 'except', 
             'very', 'several', 'among', 'being', 'be', 'mine', 'further', 'here', 'during', 'why', 'with', 'becomes', 'about', 
             'a', 'co', 'seeming', 'due', 'wherever', 'beforehand', 'detail', 'fifty', 'becoming', 'might', 'amongst', 'my', 'empty', 
             'thence', 'thereafter', 'almost', 'least', 'someone', 'often', 'from', 'keep', 'him', 'or', 'top', 'her', 'nobody',
             'sometime', 'across', 'hundred', 'only', 'via', 'name', 'eight', 'three', 'back', 'to', 'all', 'became', 'move', 'me', 
             'we', 'formerly', 'so', 'i', 'whence', 'describe', 'under', 'always', 'himself', 'in', 'herein', 'more', 'after', 
             'themselves', 'you', 'above', 'sixty', 'them', 'hasnt', 'your', 'made', 'indeed', 'most', 'everywhere', 'fifteen', 
             'but', 'must', 'along', 'beside', 'hers', 'side', 'former', 'anyone', 'full', 'has', 'yours', 'whose', 'behind', 
             'please', 'amoungst', 'mill', 'ten', 'seemed', 'sometimes', 'should', 'over', 'take', 'each', 'same', 'rather', 'latter',
             'and', 'hereupon', 'part', 'per', 'eleven', 'ever', 'enough', 'again', 'us', 'yet', 'moreover', 'mostly', 'one', 'meanwhile',
             'whither', 'there', 'toward', 'give', 'system', 'do', 'an', 'these', 'everyone', 'towards', 'this', 'bill', 'cannot', 'un', 
             'afterwards', 'beyond', 'were', 'whether', 'well', 'another', 'below', 'first', 'upon', 'any', 'none', 'many', 'serious', 
             're', 'two', 'couldnt', 'less''a', 'about', 'above', 'across', 'after', 'afterwards', 'again', 'against', 'all', 'almost','alone',
             'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'amoungst', 'amount',
             'an', 'and', 'another', 'any', 'anyhow', 'anyone', 'anything', 'anyway', 'anywhere', 'are', 'around',
             'as', 'at', 'back', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before',
             'beforehand', 'behind', 'being', 'below', 'beside', 'besides', 'between', 'beyond', 'bill', 'both',
             'bottom', 'but', 'by', 'call', 'can', 'cannot', 'cant', 'co', 'con', 'could', 'couldnt', 'cry', 'de',
             'describe', 'detail', 'did', 'do', 'does', 'doing', 'don', 'done', 'down', 'due', 'during', 'each', 'eg',
             'eight', 'either', 'eleven', 'else', 'elsewhere', 'empty', 'enough', 'etc', 'even', 'ever', 'every', 'everyone',
             'everything', 'everywhere', 'except', 'few', 'fifteen', 'fify', 'fill', 'find', 'fire', 'first', 'five', 'for',
             'former', 'formerly', 'forty', 'found', 'four', 'from', 'front', 'full', 'further', 'get', 'give', 'go', 'had',
             'has', 'hasnt', 'have', 'having', 'he', 'hence', 'her', 'here', 'hereafter', 'hereby', 'herein', 'hereupon',
             'hers', 'herself', 'him', 'himself', 'his', 'how', 'however', 'hundred', 'i', 'ie', 'if', 'in', 'inc', 'indeed',
             'interest', 'into', 'is', 'it', 'its', 'itself', 'just', 'keep', 'last', 'latter', 'latterly', 'least', 'less',
             'ltd', 'made', 'many', 'may', 'me', 'meanwhile', 'might', 'mill', 'mine', 'more', 'moreover', 'most', 'mostly',
             'move', 'much', 'must', 'my', 'myself', 'name', 'namely', 'neither', 'never', 'nevertheless', 'next', 'nine',
             'no', 'nobody', 'none', 'noone', 'nor', 'not', 'nothing', 'now', 'nowhere', 'of', 'off', 'often', 'on', 'once',
             'one', 'only', 'onto', 'or', 'other', 'others', 'otherwise', 'our', 'ours', 'ourselves', 'out', 'over', 'own',
             'part', 'per', 'perhaps', 'please', 'put', 'rather', 're', 's', 'same', 'see', 'seem', 'seemed', 'seeming',
             'seems', 'serious', 'several', 'she', 'should', 'show', 'side', 'since', 'sincere', 'six', 'sixty', 'so', 
             'some', 'somehow', 'someone', 'something', 'sometime', 'sometimes', 'somewhere', 'still', 'such', 'system',
             't', 'take', 'ten', 'than', 'that', 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'thence', 'there',
             'thereafter', 'thereby', 'therefore', 'therein', 'thereupon', 'these', 'they', 'thickv', 'thin', 'third', 'this',
             'those', 'though', 'three', 'through', 'throughout', 'thru', 'thus', 'to', 'together', 'too', 'top', 'toward',
             'towards', 'twelve', 'twenty', 'two', 'un', 'under', 'until', 'up', 'upon', 'us', 'very', 'via', 'was', 'we',
             'well', 'were', 'what', 'whatever', 'when', 'whence', 'whenever', 'where', 'whereafter', 'whereas', 'whereby',
             'wherein', 'whereupon', 'wherever', 'whether', 'which', 'while', 'whither', 'who', 'whoever', 'whole', 'whom',
             'whose', 'why', 'will', 'with', 'within', 'without', 'would', 'yet', 'you', 'your', 'yours', 'yourself',
             'yourselves']

## Building a vocabulary of words from the given documents

In [ ]:
vocab = {}    #dictionary with unique words (key) and their freq (value)
for i in range(len(X_train)):   #ith document
    word_list = []
    for word in X_train[i][1].split():   #X_train[i][0] has file no.
        word_new  = word.strip(string.punctuation).lower()   #strip(..) removes punctuation characters from beginning and end
        if (len(word_new)>2)  and (word_new not in stopwords):  
            if word_new in vocab:
                vocab[word_new]+=1
            else:
                vocab[word_new]=1            

### Plotting a graph of no of words with a given frequency to decide cutoff drequency

In [ ]:
num_words = [0 for i in range(max(vocab.values())+1)]  # i goes till it covers all frequenicies till most freq word, num_words is a list of all possible freq
freq = [i for i in range(max(vocab.values())+1)]       # x axis
total_words = 0

for key in vocab:
    num_words[vocab[key]]+=1  # num_words[with this freq] = ? how much
    
for i in range (len(num_words)):
    total_words += num_words[i]
plt.plot(freq,num_words)
plt.axis([1, 10, 0, 20000])
plt.xlabel("Frequency")
plt.ylabel("No of words")  #no of words with each freq
plt.grid()
plt.show()

In [ ]:
cutoff_freq = 150
# For deciding cutoff frequency
num_words_above_cutoff = len(vocab)-sum(num_words[0:cutoff_freq]) 
print("Number of words with frequency higher than cutoff frequency({}) :".format(cutoff_freq),num_words_above_cutoff)

### Words with frequency higher than cutoff frequency are chosen as features

In [ ]:
# (i.e we remove words with low frequencies as they would not be significant )
features = []
for key in vocab:
    if vocab[key] >=cutoff_freq:
        features.append(key)

In [ ]:
features

In [ ]:
# To represent training data as word vector counts
X_train_dataset = np.zeros((len(X_train),len(features)))
# This can take some time to complete
for i in range(len(X_train)):
#     print(i) # Uncomment to see progress
    word_list = [ word.strip(string.punctuation).lower() for word in X_train[i][1].split()]
    for word in word_list:
        if word in features:
            X_train_dataset[i][features.index(word)] += 1   

In [ ]:
X_train_dataset

In [ ]:
# To represent test data as word vector counts
X_test_dataset = np.zeros((len(X_test),len(features)))
# This can take some time to complete
for i in range(len(X_test)):
    # print(i) # Uncomment to see progress
    word_list = [ word.strip(string.punctuation).lower() for word in X_test[i][1].split()]
    for word in word_list:
        if word in features:
            X_test_dataset[i][features.index(word)] += 1

In [ ]:
X_test_dataset

In [ ]:
Y_train_dataset = np.zeros(len(Y_train))
for i in range(len(Y_train)):
    if(Y_train[i].find("talk")):
        Y_train_dataset[i] = 1
        
Y_test_dataset = np.zeros(len(Y_test))
for i in range(len(Y_test)):
    if(Y_test[i].find("talk")):
        Y_test_dataset[i] = 1

### Evaluating on Traditional Naive Bayes Classifier Method

In [ ]:
# Implementing Multinomial Naive Bayes from scratch
class MultinomialNaiveBayes:
    
    def __init__(self):
        # count is a dictionary which stores several dictionaries corresponding to each news category
        # each value in the subdictionary represents the freq of the key corresponding to that news category 
        self.count = {}
        # classes represents the different news categories
        self.classes = None
    
    def fit(self,X_train,Y_train):
        # This can take some time to complete       
        self.classes = set(Y_train)
        for class_ in self.classes:
            self.count[class_] = {}
            for i in range(len(X_train[0])):
                self.count[class_][i] = 0
            self.count[class_]['total'] = 0
            self.count[class_]['total_points'] = 0
        self.count['total_points'] = len(X_train)
        
        for i in range(len(X_train)):
            for j in range(len(X_train[0])):
                self.count[Y_train[i]][j]+=X_train[i][j]
                self.count[Y_train[i]]['total']+=X_train[i][j]
            self.count[Y_train[i]]['total_points']+=1
    
    def __probability(self,test_point,class_):
        
        log_prob = np.log(self.count[class_]['total_points']) - np.log(self.count['total_points'])
        total_words = len(test_point)
        for i in range(len(test_point)):
            current_word_prob = test_point[i]*(np.log(self.count[class_][i]+1)-np.log(self.count[class_]['total']+total_words))
            log_prob += current_word_prob
        
        return log_prob
    
    
    def __predictSinglePoint(self,test_point):
        
        best_class = None
        best_prob = None
        first_run = True
        
        for class_ in self.classes:
            log_probability_current_class = self.__probability(test_point,class_)
            if (first_run) or (log_probability_current_class > best_prob) :
                best_class = class_
                best_prob = log_probability_current_class
                first_run = False
                
        return best_class
        
  
    def predict(self,X_test):
        # This can take some time to complete
        Y_pred = [] 
        for i in range(len(X_test)):
        # print(i) # Uncomment to see progress
            Y_pred.append( self.__predictSinglePoint(X_test[i]) )
        
        return Y_pred
    
    def score(self,Y_pred,Y_true):
        # returns the mean accuracy
        count = 0
        for i in range(len(Y_pred)):
            if Y_pred[i] == Y_true[i]:
                count+=1
        return count/len(Y_pred)

In [ ]:
clf2 = MultinomialNaiveBayes()
clf2.fit(X_train_dataset,Y_train)
Y_test_pred = clf2.predict(X_test_dataset)
our_score_test = clf2.score(Y_test_pred,Y_test)  
print("Our score on testing data :",our_score_test)
print("Classification report for testing data :-")
print(classification_report(Y_test, Y_test_pred))

In [ ]:
Y_test_pred

In [ ]:
Y_test

### Evaluating on the proposed NBTC algorithm

In [ ]:
#Defining KL divergence between two numpy arrays
def KL(a, b):
    a = np.asarray(a, dtype=np.float)
    b = np.asarray(b, dtype=np.float)
    return np.sum(np.where(b!=0, a * np.log(a / b), 0))

In [ ]:
X_train_dataset

In [ ]:
p1 = []
tot_sum,count = 0,0
for i in range(len(X_train_dataset[0])):
    count = 0
    for j in range(len(X_train_dataset)):
        count += X_train_dataset[j][i]
    p1.append(count)
         
for i in range(len(p1)):
    tot_sum += p1[i]

for i in range(len(p1)):
    p1[i] = p1[i]/tot_sum
        
print(p1)

In [ ]:
p2 = []
tot_sum2,count2 = 0,0
for i in range(len(X_test_dataset[0])):
    count2 = 0
    for j in range(len(X_test_dataset)):
        count2 += X_test_dataset[j][i]
    p2.append(count2)
         
for i in range(len(p2)):
    tot_sum2 += p2[i]

for i in range(len(p2)):
    p2[i] = p2[i]/tot_sum2
        
print(p2)

In [ ]:
kl_div = KL(p1, p2)
print(kl_div)

The KL divergence between train and test data is around 0.315

In [ ]:
#Using the empirical relation between KL divergence and gamma to find gamma value for a given KL divergence value
gamma = 0.042*(math.pow(kl_div, -2.276))

In [ ]:
gamma

In [ ]:
PDu_Dl_ = gamma/(1+gamma)
PDu_Du_ = 1 - PDu_Dl_

In [ ]:
PDu_Dl_

### Algorithm 1 as given in the paper

In [ ]:
def Algo1():
    # Every element is ith iteration of this
    # Every element has C sub_elements for each class 'c'
    # Every sub_element has sub_sub_element for every document 'd'
    
    c1 = 0
    for i in Y_train_dataset:
        if(i == 1):
            c1+=1

    p1 = c1/len(Y_train_dataset)
    p0 = 1-p1
    
    T = 10  # Number of times we run the loop, more is T, more accurate will be the model
    PDu_cId =[[[0 for j in range(2)] for k in range(len(X_train_dataset))] for i in range(T+1)] # 0th iteration value 0 (lets say)
    PDu_wIc = [[[0 for k in range(len(features))] for j in range(2)] for i in range(T+1)] # ith iteration, jth class, kth word
    PDu_c = [[0 for j in range(2)] for i in range(T+1)]   # ith iteration, jth class
    PDu_d = [[1/len(X_train_dataset) for j in range(len(X_train_dataset))] for i in range(T+1)]   # ith iteration, jth document
    
    PDu_cIDu = [[0 for j in range(2)] for i in range(T+1)]
    PDu_cIDl = [[0 for j in range(2)] for i in range(T+1)]
    nDu_w_c_Dl = [[0 for i in range(len(features))] for j in range(2)]
    nDu_w_c_Du = [[0 for i in range(len(features))] for j in range(2)]
    nDu_c_Dl = [0 for i in range(2)]
    nDu_c_Du = [0 for i in range(2)]
    
    PDu_wIc_Dl = [[0 for j in range(2)] for i in range(len(features))]
    PDu_wIc_Du = [[0 for j in range(2)] for i in range(len(features))]
    
    PDu_Du = []
    
    PDu_Dl = []
    #initializing for 0th Iteration using traditional naive bayes classifier
    PDu_c[0][0] = p0
    PDu_c[0][1] = p1
    
    
    nDu_w_c = [[0 for i in range(len(features))] for j in range(2)]
    nC = [0 for i in range(2)]
    for w in range(len(features)):
        for c in range(num_category):
            for d in range(len(X_train_dataset)):  #We estimate PDu(.) using PDl(.) by traditional naive bayes
                if(Y_train_dataset[d] == c):
                    nDu_w_c[c][w] += X_train_dataset[d][w]
                    nC[c] += 1
    
#     print(nDu_w_c)
#     print(nC)
    
    for c in range(num_category):
        for w in range(len(features)):
            PDu_wIc[0][c][w] = (1+nDu_w_c[c][w])/(len(features)+nC[c])

#     print(PDu_wIc[0])
#     print(PDu_c[0])
#     print(PDu_d)

    for t in range(1,T+1):
        # Start for Eqn 6
        print(t)
        for c in range(num_category):
            for d in range(len(X_test_dataset)):
                val1 = PDu_c[t-1][c]
                for w in range(len(X_test_dataset[d])):
                    if(X_test_dataset[d][w] > 0 and Y_test_dataset[d] == c): 
                        val1 *= PDu_wIc[t-1][c][w]
                PDu_cId[t][d][c] = val1
                
        # End for Eqn 6
        for c in range(num_category):
            # Start for Eqn 7
            #Eqn 9 start               
            for d in range (len(X_train_dataset)):
                PDu_cIDu[t][c] += PDu_cId[t][d][c]*(1/len(X_train_dataset))  #doubt

            for d in range (len(X_test_dataset)):
                PDu_cIDl[t][c] += PDu_cId[t][d][c]*(1/len(X_test_dataset))   #doubt
            #Eqn 9 end

            PDu_c[t][c] = PDu_Du_*PDu_cIDu[t][c] + PDu_Dl_*PDu_cIDl[t][c]   #PDu_Du_ is tot prob for all classes c's
            # End for Eqn 7

            #Eqn 11 start
            for w in range(len(features)):
                for c in range(num_category):
                    for d in range(len(X_train_dataset)):
                        nDu_w_c_Dl[c][w] += np.sum(X_train_dataset[d])*(X_train_dataset[d][w]/np.sum(X_train_dataset[d]))*PDu_cId[t][d][c]
                    for d in range(len(X_test_dataset)):
                        nDu_w_c_Du[c][w] += np.sum(X_test_dataset[d])*(X_test_dataset[d][w]/np.sum(X_test_dataset[d]))*PDu_cId[t][d][c]
            #Eqn 11 end


            #Eqn 12 start
            for d in range (len(X_train_dataset)):
                nDu_c_Dl[c] += np.sum(X_train_dataset[d])*PDu_cId[t][d][c]

            for d in range (len(X_test_dataset)):
                nDu_c_Du[c] += np.sum(X_train_dataset[d])*PDu_cId[t][d][c]
            #Eqn 12 end

            #Eqn 10 start
            for w in range(len(features)):
                for c in range(num_category):
                    PDu_wIc_Dl[w][c] = (1+nDu_w_c_Dl[c][w])/(len(features)+nDu_c_Dl[c])
                    PDu_wIc_Du[w][c] = (1+nDu_w_c_Du[c][w])/(len(features)+nDu_c_Du[c])
            #Eqn 10 end

            #Eqn 8 start
            for w in range(len(features)):
                PDu_wIc[t][c][w] = PDu_Du_*PDu_cIDu[t][c]*PDu_wIc_Du[w][c] + PDu_Dl_*PDu_cIDl[t][c]*PDu_wIc_Dl[w][c]
            #Eqn 8 end
    return PDu_cId[T]

In [ ]:
X_train_dataset

### Algorithm 2 as given in the paper

In [ ]:
def Algo2():
    kl_div = KL(p1, p2)
    gamma = 0.042*(math.pow(kl_div, -2.276))
    PDu_Dl_ = gamma/(1+gamma)
    PDu_Du_ = 1 - PDu_Dl_
    ans = [[0 for j in range(2)] for k in range(len(X_train_dataset))]
    ans = Algo1()
    return ans

In [ ]:
pred = Algo2()

In [ ]:
pred

In [ ]:
Y_pred_algo = []
for i in range(len(pred)):
    if(pred[i][0] > pred[i][1]):
        Y_pred_algo.append(0)
    else:
        Y_pred_algo.append(1)

In [ ]:
Y_test_dataset

In [ ]:
Y_pred_algo

In [ ]:
pred_count = 0
for i in range(len(X_test_dataset)):
    if(Y_test_dataset[i] == Y_pred_algo[i]):
        pred_count += 1
        
print(pred_count/len(X_test_dataset))